In [ ]:
#                 **TANZANIAN WATER WELLS PREDICTION**

## **1.Business understanding Problem and Objective**

### Stateholder:Tanzanian NGO

### **Overview**

#### Tanzania faces significant challenges in providing consistent access to clean water for its citizens. With an estimated population of 70.5 million in mid‑2025, Tanzania remains one of Africa’s fastest-growing nations in terms of population and resource demand.

### **Business problem**
#### The client seeks insights into patterns distinguishing **functional** versus **non-functional or repair-needed wells**, to inform drilling strategies and maintenance priorities. The goal is to help villages avoid prolonged downtime and reduce socio-economic burdens especially on women and children, who are often tasked with collecting water.

### **Project objective**
#### Use the Pump It Up dataset (featuring features like pump type, installation year, payment method, and location) to build a **classification model** that:
#### - Predict well status in a **binary setup**: functional vs. needs attention (repair or non-functional).  
#### -  Identify the key features driving pump failures (e.g., pump type, installation year, management system).

### **Project goal/value**
#### Enables targeted maintenance, directing resources to at-risk wells before they fail.#### Reduces the socioeconomic burden of water insecurity by ensuring greaterhealth, education, and gender equity outcomes.

### **Data source**
#### DrivenData. (2015). Pump it Up: Data Mining the Water Table. Retrieved [Month Day Year] from https://www.drivendata.org/competitions/7/pump-it-up-data-mining-the-water-table.



###  2.Data loading and overview
#### 2.1.import libraries
#### These libraries will support data exploration, preprocessing, modeling, and evaluation:

import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

from sklearn.preprocessing import OneHotEncoder,LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import FunctionTransformer

from sklearn.model_selection import train_test_split

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import RandomizedSearchCV, KFold

from sklearn.metrics import classification_report, accuracy_score, confusion_matrix,ConfusionMatrixDisplay

from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import plot_tree

from sklearn.ensemble import RandomForestClassifier
import warnings
from sklearn.exceptions import ConvergenceWarning

from scipy.stats import randint
from imblearn.pipeline import Pipeline
from imblearn.over_sampling import SMOTE
from sklearn.metrics import precision_recall_curve



#### 2.2 Load dataset
# train dataset
df_labels=pd.read_csv("Dataset/Training set labels.csv")
df_values=pd.read_csv("Dataset/Training set values.csv")

print("Labels:")
display(df_labels.head())

print("\nValues:")
display(df_values.head())
#merge the two datasets using id
train_df = df_values.merge(df_labels, on='id')
display(train_df.head())
print("Combined shape:", train_df.shape)

#load test set
test_df = pd.read_csv('Dataset/Test set values.csv')


#combining train and test data for cleaning and preprocessing
#Add a flag so you can split them later
train_df['source'] = 'train'
test_df['source'] = 'test'
test_df['status_group'] = np.nan  # placeholder for label column

# Combine
df= pd.concat([train_df, test_df], axis=0)

#checking columns in our data
df.columns

#get data type of columns and non-null count
df.info()
def data_types(data):

    print("Our dataset has", len( data.select_dtypes(include='number').columns),
                "numeric columns")
    
    print("and", len(data.select_dtypes(include='object').columns),
          "categorical columns")
data_types(df) 
### 3.1 Data cleaning and Preprocessing

#### a) Duplicates
#checking duplicates using a function
duplicates = []

def check_duplicates(data):

    """Function that iterates through the rows of our dataset to check whether they are duplicated or not"""
    
    for i in data.duplicated():
        duplicates.append(i)
    duplicates_values = set(duplicates)
    if(len(duplicates_values) == 1):
        print('Our Dataset has no Duplicates')

    else:
        duplicates_percentage = np.round(((sum(duplicates)/len(data)) * 100 ), 2)
        print(f'Duplicated rows constitute of {duplicates_percentage} % of our dataset')
        

check_duplicates(df)
#### b) Missing values
#checking missing values
# Get total and percentage of missing values
missing_values = df.isnull().sum()
missing_percentage = (missing_values / len(df)) * 100
missing_df = pd.DataFrame({'Missing Values': missing_values, 'Percentage': missing_percentage})
missing_df = missing_df[missing_df['Missing Values'] > 0].sort_values(by='Percentage', ascending=False)

print("Missing values in dataset:")
display(missing_df)


### Scheme_name and Scheme Management
#### The main objective of this project is to analyze data in order to predict whether water wells in Tanzania are functional or non-functional. The features scheme_name and scheme_management provide information about the names of water supply schemes and their managing entities.However, these features are not expected to be strong predictors of well functionality.
#### In addition, both columns have approximately 47% missing values, which is nearly half of the dataset. Imputing these missing values would not only be difficult but also unlikely to add significant predictive power to the model.
#### Therefore we will drop them.
#dropping scheme name & scheme management.
df.drop(['scheme_name','scheme_management'], axis=1,inplace=True)
#preview shape after dropping
df.shape
### Funder
#### The funder feature contains the names of organizations or individuals who contributed to the installation of the wells, which could offer valuable insights for future partnerships and resource allocation.
#### However, this column has approximately 6% missing values. Since the proportion is relatively small and the information could be meaningful, we will drop only the rows with missing values in this column and select only top 20 funders.
#1. Check value counts for funder
funder_counts = df['funder'].value_counts()
funder_counts

#Keep only the top 20 funders
top_n = 20
top_funders = funder_counts.nlargest(top_n).index

df['funder_reduced'] = df['funder'].where(df['funder'].isin(top_funders), 'Other')
df = df.drop(columns=['funder'])


df.shape
### Installer
#### This refers to contractors who are installing the wells.This would contribute much to our stakeholder on the best contractors to work with.
#### The  percentage of the missing values are not much ,we will therefore drop the rows with the missing values only.

##### The installer column also contains names of organizations or entities responsible for installing water wells. However, this column includes many inconsistencies due to:

- **Spelling errors** (e.g., `Distric Water Department` vs. `District Water Department`)
- **Variations in formatting** (e.g., `GOVERNMENT`, `Government`, `Governmen`)
- **Synonyms or partial names** representing the same entity

##### To clean this, we used a **dictionary mapping** approach. This method replaces all known variations and misspellings with a consistent, unified label.
# Dictionary of common replacements for installer names
installer_mapping = {
    # District Council
    'District Water Department': 'District water department',
    'District water depar': 'District water department',
    'Distric Water Department': 'District water department',
    
    # Fini Water
    'FinW': 'Fini Water',
    'Fini water': 'Fini Water',
    'FINI WATER': 'Fini Water',
    
    # Jaica
    'JAICA': 'Jaica',
    
    # District Council Variants
    'COUN': 'District council',
    'District COUNCIL': 'District council',
    'DISTRICT COUNCIL': 'District council',
    'District Counci': 'District council',
    'District Council': 'District council',
    'Council': 'District council',
    'Counc': 'District council',
    'District  Council': 'District council',
    'Distri': 'District council',
    
    # RC Church Variants
    'RC CHURCH': 'RC Church',
    'RC Churc': 'RC Church',
    'RC': 'RC Church',
    'RC Ch': 'RC Church',
    'RC C': 'RC Church',
    'RC CH': 'RC Church',
    'RC church': 'RC Church',
    'RC CATHORIC': 'RC Church',
    
    # Central Government Variants
    'Central Government': 'Central government',
    'Tanzania Government': 'Central government',
    'central government': 'Central government',
    'Cental Government': 'Central government',
    'Cebtral Government': 'Central government',
    'Tanzanian Government': 'Central government',
    'Tanzania government': 'Central government',
    'Centra Government': 'Central government',
    'CENTRAL GOVERNMENT': 'Central government',
    'TANZANIAN GOVERNMENT': 'Central government',
    'Central govt': 'Central government',
    'Centr': 'Central government',
    'Centra govt': 'Central government',
    
    # World Vision
    'World vision': 'World Vision',
    'World Division': 'World Vision',
    
    # Unicef
    'Unisef': 'Unicef',
    'UNICEF': 'Unicef',
    
    # DANIDA
    'DANID': 'DANIDA',
    
    # Villagers
    'villigers': 'villagers',
    'villager': 'villagers',
    'Villagers': 'villagers',
    'Villa': 'villagers',
    'Village': 'villagers',
    'Villi': 'villagers',
    'Village Council': 'villagers',
    'Village Counil': 'villagers',
    'Villages': 'villagers',
    'Vill': 'villagers',
    'Village community': 'villagers',
    'Villaers': 'villagers',
    'Village Community': 'villagers',
    'Villag': 'villagers',
    'Villege Council': 'villagers',
    'Village council': 'villagers',
    'Village  Council': 'villagers',
    'Villagerd': 'villagers',
    'Villager': 'villagers',
    'Village Technician': 'villagers',
    'Village Office': 'villagers',
    'Village community members': 'villagers',
    
    # Community
    'Commu': 'Community',
    'Communit': 'Community',
    'commu': 'Community',
    'COMMU': 'Community',
    'COMMUNITY': 'Community',
    
    # Government Variants
    'GOVERNMENT': 'Government',
    'GOVER': 'Government',
    'GOVERNME': 'Government',
    'GOVERM': 'Government',
    'GOVERN': 'Government',
    'Gover': 'Government',
    'Gove': 'Government',
    'Governme': 'Government',
    'Governmen': 'Government',
    
    # Other custom fixes
    'Hesawa': 'HESAWA',
    'Colonial Government': 'Colonial government',
    'Government of Misri': 'Misri Government',
    'Italy government': 'Italian government',
    'British colonial government': 'British government',
    'Concern /government': 'Concern/Government',
    'Village Government': 'Village government',
    'Government and Community': 'Government /Community',
    'Cetral government /RC': 'RC church/Central Gover',
    'Government /TCRS': 'TCRS /Government',
    'Government/TCRS': 'TCRS /Government',
    'ADRA /Government': 'ADRA/Government'
}

# Apply the replacements
df['installer'] = df['installer'].replace(installer_mapping)

### Public Meeting
#### Public meetings are a way of the community to come together and raise issues of concern. We will also just drop missing values of the column.
#drop missing values in funder,installer,public meeting
df = df.dropna(subset=['funder_reduced', 'installer', 'public_meeting'])
#preview shape
df.shape
## 4.Exploratory Data Analysis (EDA)
### 4.1 Binary Target Variable Creation and Class Distribution
#### To simplify the prediction task, have converted the original status_group target variable into a binary classification problem as per the objectives. Wells labeled as "functional" remain as is, while those labeled as "needs_repair" or "non-functional" were combined into a single "needs_attention" class.
# Create binary target variable
df['status_binary'] = df['status_group'].apply(lambda x: 'functional' if x == 'functional' else 'needs_attention')



#### Class Distribution
# Count how many rows in each class
class_counts = df['status_binary'].value_counts()
print("Number of wells per class:")
print(class_counts)

# Display the proportion (percentage) per class
class_proportions = df['status_binary'].value_counts(normalize=True) * 100
print("\nPercentage distribution per class:")
print(class_proportions.round(2))

# Plot the class distribution
class_counts.plot(kind='bar', color=['green', 'red'], title='Binary Target Class Distribution')
plt.xlabel('Well Status')
plt.ylabel('Number of Wells')
plt.xticks(rotation=0)
plt.show()
#### The dataset is fairly balanced with respect to the binary target classes:This shows that:
#### About 56.44% of wells are functional
#### About 43.56% need attention
#### This near-even split ensures that our model will have sufficient examples from both classes to learn meaningful patterns, reducing the risk of bias towards the majority class.
### 4.2 **Univariate Analysis (Categorical Features)**
#### Exploring key categorical variables to understand their distributions.
# visualizing Catogorical variables
# Drop ID column
if 'id' in df.columns:
    df = df.drop(columns=['id'])

# Categorical columns to visualize
categorical_cols = ['funder_reduced', 'installer', 'payment_type', 'water_quality', 'source_type']

# Setup the subplot grid
num_plots = len(categorical_cols)
cols = 2  
rows = (num_plots + cols - 1) // cols
fig, axes = plt.subplots(rows, cols, figsize=(14, 5 * rows))
axes = axes.flatten()  # Flatten to 1D array for easy indexing

for i, col in enumerate(categorical_cols):
    top10 = df[col].value_counts().iloc[:10].index
    sns.countplot(data=df, x=col, order=top10, ax=axes[i], palette='Set2')
    axes[i].set_title(f'Distribution of {col}')
    axes[i].set_xlabel("")
    axes[i].tick_params(axis='x', rotation=90)
plt.tight_layout()
plt.show()
#### From the visualization of the categorical columns,it is clear that:
#### - **Funder:** Government of Tanzania funds the majority of wells.
#### - **Installer:** DWE is the most frequent installer — their wells' performance may be significant.
#### - **Payment Type:** Most wells are community-funded (free), suggesting non-commercial purpose.
#### - **Water Quality:** Majority use soft water relevant for assessing water safety.
#### - **Source Type:** Shallow wells, springs, and boreholes dominate, potentially influencing functionality.

### 4.3 Distribution of Numerical values
###  Checking outliers 
#checking outliers in numerical data using Interquartile range
#select numerical columns only
numerical_data=df.select_dtypes(include='number')


for col in numerical_data.columns:
    Q1 = numerical_data[col].quantile(0.25)
    Q3 = numerical_data[col].quantile(0.75)
    IQR = Q3 - Q1

    # Calculate bounds for outliers
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR

    # Extract outlier rows
    outliers = df[(numerical_data[col] < lower_bound) | (numerical_data[col] > upper_bound)]

    print(f"Column: {col}")
    if not outliers.empty:
        print(outliers[[col]])
    else:
        print("No outliers found.")
    print("-" * 50)
#Visualizating the outliers in our numerical data
def plt_boxplots(data,cols):
    fig, axes =plt.subplots(2,4, figsize=(20,10))
    axes=axes.ravel()
    

    colors=['black','skyblue','green','blue','black','purple','pink','brown','gray']

    for i,col in enumerate(cols[:8]):
        # convert the x-axis variable to a numeric data type
        
        data[col] = data[col].astype(float)
        sns.boxplot(x=data[col], ax=axes[i], color=colors[i])
        
        axes[i].set_title(col)
    plt.tight_layout()
cols = df.select_dtypes(include='number').columns

plt_boxplots(df, cols)


##### **Amount_tsh**-Highly skewed with many extreme outliers on the right.Most of the data concentrated near zero.outliers to be handled or use log transformation.
##### **gps height**-More symmetric distribution, but still shows some right-side outliers.
##### **construction year**-Relatively well-distributed with no strong outliers.Appears normally distributed with a peak near a central value.
##### **Population**-Very right-skewed — many outliers in high population areas.Most wells serve a small population, with a few outliers serving large communities.
##### **num_plate**-Highly skewed and sparse — almost all values are 0 or low.Presence of extreme outliers might be a low-importance feature.
##### **Region code** and **district code**-Both appear more categorical than numeric.to be ploted as barplot.


#### 4.3b Irrelevant columns
##### The following columns appear not to have relevant infomation towards our project,we will therefore  drop them:(**Permit,Subvillage,wpt_name,region_code,district_code,iga, ward,recorded_by and date_recorded**)

irrelevant_columns = ['permit', 'subvillage', 'wpt_name', 'region_code', 'district_code',
                      'lga', 'ward', 'recorded_by', 'date_recorded','num_private']

# Keep only those columns that are actually still in the DataFrame
columns_to_drop = [col for col in irrelevant_columns if col in df.columns]

# Drop the invalid columns
df.drop(columns=columns_to_drop, inplace=True)

print(f"Dropped columns: {columns_to_drop}")
print(f"Current shape: {df.shape}")
### 4.4 Relationship between Target variable and features
#### Categorical vs Target variable

#Layout
n_cols =2
n_rows=(len(categorical_cols)+n_cols-1) // n_cols

# create figure
fig,axes=plt.subplots(n_rows, n_cols,figsize=(16,5 * n_rows))
axes=axes.flatten()

for i,col in enumerate(categorical_cols):
    top10 = df[col].value_counts().iloc[:10].index
    sns.countplot(data=df, x=col, hue='status_binary', order=top10, ax=axes[i], palette='Set2')
    axes[i].set_title(f'{col} vs Well Status')
    axes[i].tick_params(axis='x', rotation=90)
    axes[i].set_xlabel('')
    axes[i].set_ylabel('Count')
plt.tight_layout()
plt.show()
#### From the visualization ,relationship between categorical vs target is as follows:
#### **Funder vs status** -Government of Tanzania funds the largest number of wells, but a higher portion of these wells need attention than are functional. Danida, Rwssp, and World Vision show a more balanced distribution, with relatively more functional wellsHesawa, Kkkt, and World Bank have a higher number of wells needing attention.
#### **Installer vs status**-DWE is by far the most common installer, with a large number of wells both functional and needing attention.
#### **Payment vs status**-"Never pay" is the most common and has high failure."Per bucket" and "monthly" payments align more with functional wells, possibly reflecting better maintenance via community investment.
### **water Quality vs Status**-Wells with soft water are more likely to be functional.wells with salty or milky water show higher need for attention.
#### **Source type vs status** -Spring and river/lake sources show better functionality.Shallow wells and boreholes are more prone to needing attention.


### 4.5 Numerical vs Target variable
# Select only key numeric features for comparison
selected_numeric_cols = ['amount_tsh', 'gps_height', 'population', 'construction_year']
# layout
num_features = len(selected_numeric_cols)
cols_per_row = 3
rows = (num_features + cols_per_row - 1) // cols_per_row

plt.figure(figsize=(18, 5 * rows))

for i, col in enumerate(selected_numeric_cols, 1):
    plt.subplot(rows, cols_per_row, i)
    sns.boxplot(data=df, x='status_binary', y=col, palette='Set2')
    plt.title(f'{col} vs Well Status')
    plt.xlabel('Well Status')
    plt.ylabel(col)

plt.tight_layout()
plt.show()

#### **
### 5.Preprocessing

### 5.1 Log transformation for skewed numerical features
#### amount_tsh and population are highly skewed.applying log transformation will compress the scale for a better model.
# Log transform skewed features
df['amount_tsh_log'] = np.log1p(df['amount_tsh'])
df['population_log'] = np.log1p(df['population'])

#Drop original amount_tsh and population
df.drop(['amount_tsh', 'population'], axis=1, inplace=True)

### 5.3 Construction year to Well Age
##### The well_age feature represents the age of a water well calculated as the difference between the current year (2025) and the year the well was constructed. This helps us understand how the age of a well affects its functionality, as older wells might be more prone to failures.
##### Since some wells have invalid construction years (e.g. 0), we replace these with missing values and then impute using the median well age to maintain data integrity.
# Calculate well age
df['well_age'] = 2025 - df['construction_year']

# Handle invalid years (e.g., 0)
df.loc[df['well_age'] == 2025, 'well_age'] = np.nan  # Replace invalid '0' year wells with NaN

# Impute missing well_age with median value
df['well_age'] = df['well_age'].fillna(df['well_age'].median())
### 5.2 Encode Categorical Variables
##### Machine learning model will require numerical inputs.We will use one-hot encoding to convert categorical features into binary.To avoid dummy variable trap, we will drop the first category from each one-hot encoded feature using drop_first=True
#identify our cleaned categorical columns exclude the target.
categ_cols=cat_cols = df.select_dtypes(include='object').columns.tolist()
categ_cols = [col for col in categ_cols if col not in ['status_group', 'status_binary']]
print("Initial categorical columns:")
print(categ_cols)

# Encode selected categorical variables using One-Hot Encoding
df_encoded = pd.get_dummies(df,categ_cols, columns=categ_cols, drop_first=True)
print(f"Shape after encoding: {df_encoded.shape}")
### 5.3 Split dataset back to Train and Test
# Separate train and test based on the 'source' column
train_cleaned = df_encoded[df_encoded['source_train'] == 1].copy()
test_cleaned = df_encoded[df_encoded['source_train'] == 0].copy()

# Drop the source columns
train_cleaned.drop(columns=['source_train'], inplace=True)
test_cleaned.drop(columns=['source_train', 'status_group', 'status_binary'], errors='ignore')
train_cleaned.columns
### 6. Modeling
#### 6.1 Create features and target
##### X to be our features and y to be our target
# Features and target
X = train_cleaned.drop(columns=['status_group', 'status_binary'], errors='ignore')
y = train_cleaned['status_binary']

#### 6.2 Train/Test split for Validation
##### Split data and Use a portion of the training set to validate the model
#use test size of 0.2 and random state 42

X_train, X_val, y_train, y_val = train_test_split( X, y, stratify=y, test_size=0.2, random_state=42)

### 6.3 Standardization
##### in our dataset, several numerical feature exist on **different scales** we will standardize them using standardscaler:

- `amount_tsh_log` might range between 0 and 15
- `gps_height` can span from -50 to over 2000
- `well_age` ranges from 0 to over 60 years
- `population_log` values vary depending on the community size
# Instantiate the scaler
scaler=StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled = scaler.transform(X_val)

### 6.4 Logistic Model
#### will develop a logistic model to be my  first base model.
#  Instantiate the model
log_model = LogisticRegression(max_iter=300,solver='saga', random_state=42)
#  Fit the model
log_model.fit(X_train_scaled, y_train)
#  Make predictions
y_pred = log_model.predict(X_val_scaled)
warnings.filterwarnings("ignore", category=ConvergenceWarning)

# 4. Evaluate performance
print("=== Confusion Matrix ===")
#print(confusion_matrix(y_val, y_pred))

print("\n=== Classification Report ===")
print(classification_report(y_val, y_pred))

print("\n=== Accuracy Score ===")
print("Accuracy:", round(accuracy_score(y_val, y_pred) * 100, 2), "%")
#Visualize our results
# Get predictions and confusion matrix
y_pred = log_model.predict(X_val_scaled)
cm = confusion_matrix(y_val, y_pred, labels=["functional", "needs_attention"])

# Plot using seaborn heatmap
plt.figure(figsize=(6,4))
sns.heatmap(cm, annot=True, fmt='d', cmap='YlGnBu', 
            xticklabels=["functional", "needs_attention"], 
            yticklabels=["functional", "needs_attention"])

plt.xlabel("Predicted Label")
plt.ylabel("True Label")
plt.title("Confusion Matrix")
plt.tight_layout()
plt.show()
##### True Positives (TP): 4900 Wells that are functional and were correctly predicted as functional.

#### False Negatives (FN): 837 Wells that are functional, but were predicted as needing attention.These are missed good wells ,could lead to unnecessary maintenance.

#### False Positives (FP): 1532 Wells that need attention, but were predicted as functional.This is a serious concern because these wells may fail and get overlooked.

#### True Negatives (TN): 3256 Wells that need attention, and were correctly predicted.
### Visualize metrics-Accurancy,recall,precision and F1Score
# Get classification report as dict
report = classification_report(y_val, y_pred, output_dict=True)
report_df = pd.DataFrame(report).transpose()

# Check the columns available
print(report_df.columns)
plt.figure(figsize=(8, 5))
sns.heatmap(report_df.loc[['functional', 'needs_attention'], ['precision', 'recall', 'f1-score']], annot=True, cmap='YlGnBu', fmt='.2f')
plt.title('Classification Report Metrics Heatmap')
plt.show()
##### as per our visualiztion of metrics :
#### Precision = 0.80 (80% of predicted wells needing attention were correct)

##### Recall = 0.68 (68% of actual wells needing attention were found)

##### F1-score = 0.73 (balance of the two)
#### 6.5 Decision trees
##### will develop decision tree model as my secod base model
# Initialize model with some parameters to control overfitting
dt_model = DecisionTreeClassifier(random_state=42, max_depth=10, min_samples_leaf=10)

# Train model
dt_model.fit(X_train, y_train)

# Predict on validation set
y_pred_dt = dt_model.predict(X_val)

# Evaluate results
print(classification_report(y_val, y_pred_dt))
#visualize the decision tree
plt.figure(figsize=(20, 10))  # Adjust size as needed
plot_tree(
    dt_model,
    feature_names=X_train.columns,
    class_names=dt_model.classes_,
    filled=True,
    max_depth=3,  # Limit depth shown for readability
    fontsize=10
)
plt.title("Decision Tree Visualization (Depth=3)")
plt.show()
##### as per the decision tree model:
##### **Functional**
##### Recall = 0.87: The model correctly identifies 87% of truly functional wells.
##### Precision = 0.74: Out of all predicted functional wells, 74% were actually functional.

##### **Needs Attention**
##### Recall = 0.64: The model is missing 36% of the "needs_attention" wells (false negatives).
##### Precision = 0.81: When it predicts "needs_attention", it's right 81% of the time — pretty good.

##### **Overall Accuracy: 77%**
##### That’s strong and comparable to many real-world models, especially for imbalanced classes.
#### 6.5 **Random forest model**
##### We will develop Random forest model as our third base model
# 1. Initialize model
rf_model = RandomForestClassifier(
    n_estimators=100,
    max_depth=None,
    random_state=42,
    n_jobs=-1
)
# 2. Train model
rf_model.fit(X_train_scaled, y_train)

# 3. Predict
y_pred_rf = rf_model.predict(X_val_scaled)

# 4. Evaluate
print("=== Confusion Matrix ===")
print(confusion_matrix(y_val, y_pred_rf))

print("\n=== Classification Report ===")
print(classification_report(y_val, y_pred_rf))
print("\n=== Accuracy Score ===")
print(f"Accuracy: {accuracy_score(y_val, y_pred_rf) * 100:.2f} %")
##### The Random Forest classifier achieved an overall accuracy of 81.87% on the validation set. For the functional class, the model reached a precision of 0.82, a recall of 0.86, and an F1-score of 0.84, based on 5,737 samples. For the needs attention class, it obtained a precision of 0.82, a recall of 0.77, and an F1-score of 0.79, evaluated on 4,788 samples.
#### 6.6 **Comparison for the three Models**
##### Logistic Regression, as the first base model, achieved an overall accuracy of 77.49%. It performed well on the functional class with a recall of 85%, but struggled with the needs attention class, achieving only 68% recall. This indicates that while it is effective at identifying working waterpoints, it tends to miss a significant number of those needing attention.

##### Decision Tree, the second base model, had a slightly lower accuracy of 77.0%. It achieved a high recall of 87% for functional waterpoints but only 64% recall for needs attention. This suggests it overemphasized the majority class, making it prone to overfitting and less reliable for detecting waterpoints in need of repair.

##### Random Forest, the third base model, outperformed both previous models with an accuracy of 81.87%. It maintained a strong balance between precision and recall across both classes achieving 86% recall for functional and 77% recall for needs attention. This makes it the most robust and reliable model among the three for classifying waterpoint functionality
#### 7. Hyperparameter Tuning
##### From the comparison of the three base models , Random forest stood out therefore we will tune it further by balancing the class weights between functional and needs attendion.

# Initialize Random Forest with balanced class weights
rf_model = RandomForestClassifier(n_estimators=100, class_weight='balanced', random_state=42)

# Fit the model
rf_model.fit(X_train, y_train)

# Predict on the validation set
y_pred = rf_model.predict(X_val)

# Evaluate performance
from sklearn.metrics import classification_report
print(classification_report(y_val, y_pred))


##### Both classes attain equal precision (0.82), indicating reliability in positive predictions.
#### Recall for functional (0.86) is strong, meaning the majority of working wells are identified.
#### Recall for needs_attention (0.77) is slightly lower, meaning 23% of failing wells are missed.
#### The F1-scores (0.84 and 0.79) show a good balance between precision and recall.
#### Overall, the model achieves 82% accuracy, with macro and weighted averages supporting class balance
#### 7.1 ROC AND AUC  ,Visualization  of Random Forest model
from sklearn.metrics import roc_curve, roc_auc_score
# Get probability estimates for the positive class ("needs_attention")
y_probs = rf_model.predict_proba(X_val_scaled)[:, 1]

# Compute false positive rate (FPR), true positive rate (TPR), and thresholds
fpr, tpr, thresholds = roc_curve(y_val, y_probs, pos_label='needs_attention')

# Compute the AUC
roc_auc = roc_auc_score(y_val, y_probs)
print(f"Validation ROC AUC: {roc_auc:.3f}")
#Visualize
plt.figure(figsize=(8,6))
plt.plot(fpr, tpr, label=f'ROC curve (AUC = {roc_auc:.3f})', lw=2)
plt.plot([0,1], [0,1], 'k--', label='Random baseline')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curve-Random Forest')
plt.legend(loc='lower right')
plt.grid(True)
plt.show()
##### Results
##### Having a ROC of 0.84 Shows that our model is doing good being less than 0.5
#### AUC represents the probability that your model will assign a higher “needs_attention” score to a truly failing well than to a functional one.

#### At 0.814, this implies there's about an 81.4% chance a randomly sampled failing well will be ranked above a functional one well above random chance (50%)